In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from messages import messages
from binascii import unhexlify

In [3]:
msg = messages[4]

In [123]:
# split content
structure = {
    'start byte': 2,
    'LENGTH system title': 0,
    'system title': 8,
    'seperator': 1,
    'LENGTH of subsequent bytes': 0,
}

start = 0
end = 2
print(f"Start 0x{msg[start:end]}")

start = end
end = start + 2
l_system_title = int(msg[start:end], 16)
print(f"Length system title: {l_system_title}")

# System Title:
start = end
end = start + l_system_title*2
system_title = msg[start:end]
print(f"System title {unhexlify(system_title)}")

start = end
end = start + 2
print(f"Separator 0x{msg[start:end]}")

# Data length
start = end
end = start + 4
l_data = int(msg[start:end], 16)
print(f"Length {l_data}")

# seperator 0x30
start = end
end = start + 2
print(f"Separator: 0x{msg[start:end]}")

# frame counter
start = end
end = start + 4*2
frame_counter = msg[start:end]
print(f"Frame counter: {int(frame_counter,16)}")

# cipher frame
start = end
end = start + 2*(l_data - 5 - 12)
frame = msg[start:end]

# GCM TAG
start = end
end = start + 12*2
gcm_tag = msg[start:end]
print(f"GCM Tag: {gcm_tag}")

Start 0xdb
Length system title: 8
System title b'SAG5\x00\x01\xd1\x7f'
Separator 0x82
Length 498
Separator: 0x30
Frame counter: 5731
GCM Tag: 44fa2040d4887b71a19fe5a5


In [124]:
from Crypto.Cipher import AES

encryption_key = unhexlify("98B362EF72DC06043829C1AAE16A6562")
init_vector = unhexlify(system_title + frame_counter)
cipher = AES.new(encryption_key, AES.MODE_GCM, nonce=init_vector)

In [125]:
bframe = unhexlify(frame)
apdu_complete = cipher.decrypt(bframe)
apdu = apdu_complete#.split(b'!')[0]
print(apdu)

b'/EST5\\253740976_A\r\n\r\n1-3:0.2.8(50)\r\n0-0:1.0.0(220815084916S)\r\n1-0:1.8.0(008995832*Wh)\r\n1-0:1.8.1(007130300*Wh)\r\n1-0:1.8.2(001865532*Wh)\r\n1-0:1.7.0(000000465*W)\r\n1-0:2.8.0(000000112*Wh)\r\n1-0:2.8.1(000000000*Wh)\r\n1-0:2.8.2(000000112*Wh)\r\n1-0:2.7.0(000000000*W)\r\n1-0:3.8.0(000030668*varh)\r\n1-0:3.8.1(000028978*varh)\r\n1-0:3.8.2(000001690*varh)\r\n1-0:3.7.0(000000036*var)\r\n1-0:4.8.0(004045755*varh)\r\n1-0:4.8.1(002828123*varh)\r\n1-0:4.8.2(001217632*varh)\r\n1-0:4.7.0(000000250*var)\r\n!B766\r\n'


In [21]:
from dsmr_parser.parsers import TelegramParser
from dsmr_parser.objects import Telegram
from dsmr_parser import telegram_specifications
from messages import telegrams

In [36]:
parser = TelegramParser(telegram_specifications.V5)

In [37]:
t = Telegram(telegrams[0].decode('ASCII'), parser ,telegram_specifications.V5 )
print(t)

P1_MESSAGE_HEADER: 	 50	[None]
P1_MESSAGE_TIMESTAMP: 	 2022-08-21T16:54:23+02:00	[None]
ELECTRICITY_IMPORTED_TOTAL: 	 9053098	[Wh]
ELECTRICITY_USED_TARIFF_1: 	 7173764	[Wh]
ELECTRICITY_USED_TARIFF_2: 	 1879334	[Wh]
ELECTRICITY_DELIVERED_TARIFF_1: 	 0	[Wh]
ELECTRICITY_DELIVERED_TARIFF_2: 	 112	[Wh]
CURRENT_ELECTRICITY_USAGE: 	 413	[W]
CURRENT_ELECTRICITY_DELIVERY: 	 0	[W]



In [19]:
tg = parser.parse(telegrams[0].decode('ASCII'))
print(tg)
for key in tg:
    t = Telegram(parser, tg[key],telegram_specifications.V2_2 )
    print(t)

{'\\d-\\d:1\\.8\\.1.+?\\r\\n': <dsmr_parser.objects.CosemObject object at 0x10a8cbc70>, '\\d-\\d:1\\.8\\.2.+?\\r\\n': <dsmr_parser.objects.CosemObject object at 0x10a8d7970>, '\\d-\\d:2\\.8\\.1.+?\\r\\n': <dsmr_parser.objects.CosemObject object at 0x10a8d6b30>, '\\d-\\d:2\\.8\\.2.+?\\r\\n': <dsmr_parser.objects.CosemObject object at 0x10a8d7a90>, '\\d-\\d:1\\.7\\.0.+?\\r\\n': <dsmr_parser.objects.CosemObject object at 0x10a8d4850>, '\\d-\\d:2\\.7\\.0.+?\\r\\n': <dsmr_parser.objects.CosemObject object at 0x10a8d79d0>}


AttributeError: 'CosemObject' object has no attribute 'parse'